In [1]:
import pandas_datareader.data as web #to collect data
import datetime as dt #to specify start and end dates

# import yfinance as yf

import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import scipy.stats as stats
from scipy.spatial.distance import cdist


from sklearn.neighbors import NearestNeighbors

import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

from patsy import dmatrices
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Data Reading

In [2]:
car = pd.read_pickle("Director Level CAR_on_appointment_2 v170325.pkl").rename({"Symbol_x":"Symbol", "AsOnDate_x":"AsOnDate"}, axis = 1)

### Filtering dates with no Indep classification

In [3]:
car2 = car.loc[ pd.notnull(car["Independent (Yes/No)"])].reset_index(drop = True)
car2

,Symbol,Company,AsOnDate,AsOnYear,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,Prowess company code,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,NSE symbol,Head office address,Skills,EducationAll,IsMBA,IsPhD,AllPastDirectNIC,CurrDirectANIC,CurrDirectABNIC,AllNIC,AllNIC_Industry,HasFinanceXP,HasTechXP,HasRelatedIndustryXP,HasExecXP,PublicExecXPDummy,PrivateExecXPDummy,HasPublicExecXP,HasPrivateExecXP,SkillsInPositiononBoard,SkillsInOccupation,AllSkills,HasTechSkill,HasFinanceSkill,FirmsPast,NumSkills,NumFirmsPast,NumIndustryPast,HasCeoMDChairXP,HasConglomerateXP,General Ability Index,ProwessCode,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11
0,3MINDIA,3M INDIA LTD.,2006-03-31,2006,INE470A01017,P00033272,MR.,SHANKARANARAYANA,VISHWANATHA RAO,BANAWARA,1960-05-14,M,INDIA,NONE,NaN,nan,nan,GRADUATE: B.COM.,"POST GRADUATE: M.B.A.-FINANCE,BANGALORE UNIVER...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ACCOUNTS/AUDIT/TAXATION,FINANCE,LEADERSHIP/MAN...",nan,nan,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"MR. B.V. SHANKARANARAYANA RAO, 57, HOLDS A BAC...",NaT,U,2002-07-24,NaT,NaT,2007-03-31,0,0.0,1.0,1.0,0.0,0.0,0.0,1346.0,0.0,0.0,981.0,0.0,0.0,981.0,1346.0,0.0,0.0,1346.0,981.0,0.0,0.0,981.0,1346.0,0.0,0.0,1346.0,1345,(),"{(3MINDIA, U)}","{(3MINDIA, U)}",(

## Financial Data Wrangle and Merge

In [4]:
fin = pd.read_pickle("Main_Firm_No Loc_Fin v170325.pkl")

In [5]:
fin.columns.to_list()

['FirmID',
 'Symbol',
 'AsOnDate',
 'TotDirCount',
 'AsOnYear',
 'LnBoardSize',
 'FemaleDirCount',
 'PercentWomenDir',
 'Women_dummy',
 'AvgAge',
 'CountRookie',
 'Rookie_dummy',
 'CountNonRookie',
 'CountIndep',
 'PercentIndep',
 'CountNonIndep',
 'CountRookieIndep',
 'CountRookieNonIndep',
 'CountNonRookieIndep',
 'CountNonRookieNonIndep',
 'CountPromoterDir',
 'CountBusyDir',
 'PercentBusyDir',
 'Busy_dummy',
 'HasCeoMD',
 'HasChairman',
 'HasPromoterOnBoard',
 'HasDualityChairmanMD',
 'IsFamilyManaged',
 'HasFamilyChairman',
 'HasFamilyChairmanAndCEO',
 'CountFirstTermDir',
 'CountFirstTermIndepDir',
 'PercentFirstTermIndepDir',
 'CountTurnOverIndepDir',
 'HasIndepTurnOver',
 'AvgTenureInYearsinCompTotal',
 'AvgTenureInYearsinCompIndep',
 'PercentRookieIndep',
 'PercentNonRookieIndep',
 'PercentRookieNonIndep',
 'PercentNonRookieNonIndep',
 'PercentRookie',
 'PercentNonRookie',
 'IsRookieBoard',
 'JustOneRookie',
 'TwoOrMoreRookies',
 'CountMBA',
 'PercentMBA',
 'CountIndepMBA',
 '

In [6]:
finCol = fin.columns[[0, 1, 2, 4]].append(fin.columns[145:])
finCol

Index(['FirmID', 'Symbol', 'AsOnDate', 'AsOnYear', 'Company Name', 'Sales', 'Profit after tax', 'PBDITA', 'Return on total assets', 'Total assets', 'Operating  profit of non-financial companies', 'Operating profit of financial companies', 'Debt to equity ratio (times)', 'Long term borrowings incl current portion', 'Long term borrowings excl current portion', 'Debt', 'Trade payables (Old Sch. VI)', 'Subscribed preference capital', 'Paid up preference capital (net of forfeited preference capital) (Old Sch. VI)', 'Paid up preference capital (net of forfeited preference capital)', 'Paid up preference shares', 'Total other receivables', 'Net cash flow from operating activities', 'Net cash inflow or (outflow) from investing activities', 'Cash inflow due to sale of fixed assets', 'Cash (outflow) due to purchase of fixed assets', 'Cash inflow due to sale of Intangible assets', 'Cash (outflow) due to purchase of Intangible assets', 'Research & development expenses',
       'Inter-corporate loan

### Data Wrangle

In [7]:
fin2 = fin.loc[:, finCol]
fin2[["Symbol", "AsOnDate"]].drop_duplicates().sort_values(by = ["Symbol", "AsOnDate"])

fin2["log_Market Capitalisation"] = np.log(fin2["Market Capitalisation"])
fin2["log_Total assets"] = np.log(fin2["Total assets"])

fin2 = fin2.replace([np.inf, -np.inf], np.nan)

C:\Users\SHIVAM\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\SHIVAM\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
car2["NIC code"]

0         22209
1         22209
2         29102
3         64300
4         23912
          ...  
259413    21002
259414    10409
259415    10409
259416    10409
259417    10409
Name: NIC code, Length: 259418, dtype: object

In [9]:
fin2["NIC code"]

0           08
1           08
2           08
3           08
4           08
         ...  
28904    62013
28905    62013
28906    62013
28907    62013
28908    62013
Name: NIC code, Length: 28909, dtype: object

### Merge

In [10]:
carFin = car2.merge(fin2, on = ["Symbol", "AsOnDate", "Entity type", "Entity type code", "Incorporation year", "Industry group",
                             "Industry group code", "NIC name", "NIC code", "Ownership group code", "Ownership group", "Age group",
                             "Head office address"], how = "inner")
carFin

,Symbol,Company,AsOnDate,AsOnYear_x,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,Prowess company code_x,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,NSE symbol_x,Head office address,Skills,EducationAll,IsMBA,IsPhD,AllPastDirectNIC,CurrDirectANIC,CurrDirectABNIC,AllNIC,AllNIC_Industry,HasFinanceXP,HasTechXP,HasRelatedIndustryXP,HasExecXP,PublicExecXPDummy,PrivateExecXPDummy,HasPublicExecXP,HasPrivateExecXP,SkillsInPositiononBoard,SkillsInOccupation,AllSkills,HasTechSkill,HasFinanceSkill,FirmsPast,NumSkills,NumFirmsPast,NumIndustryPast,HasCeoMDChairXP,HasConglomerateXP,General Ability Index,ProwessCode,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11,FirmID,AsOnYear_y,Company Name,Sales,Profit after tax,PBDITA,Return on total assets,Total assets,Operating profit of non-financial companies,Operating profit of financial companies,Debt to equity ratio (times),Long term borrowings incl current portion,Long term borrowings excl current portion,Debt,Trade payables (Old Sch. VI),Subscribed preference capital,Paid up preference capital (net of forfeited preference capital) (Old Sch. VI),Paid up preference capital (net of forfeited preference capital),Paid up preference shares,Total other receivables,Net cash flow from operating activities,Net cash inflow or (outf

#### Verifying a 1 to 1 merge

In [11]:
diff = carFin.sort_values(by=["Symbol", "AsOnDate", "Person Code"])\
    .merge(car2, left_index = True, right_index = True, how="outer", indicator=True)\
    .query('_merge != "both"')\
    .drop(columns=['_merge'])
diff

,Symbol_x,Company_x,AsOnDate_x,AsOnYear_x,ISIN_x,Person Code_x,Director Salutation_x,Director First Name_x,Director Middle Name_x,Director Surname_x,Date of Birth_x,Gender_x,Nationality_x,Member of Civil Services_x,Promoter Director (Yes/No)_x,Position on Board_x,Independent (Yes/No)_x,Education1_x,Education2_x,Education3_x,Education4_x,Education5_x,Education6_x,Education7_x,Education8_x,Education9_x,Education10_x,Skills/Competencies_x,Occupation_x,Cessation Reason_x,Other Directorship 1_x,Other Directorship 2_x,Other Directorship 3_x,Other Directorship 4_x,Other Directorship 5_x,Other Directorship 6_x,Other Directorship 7_x,Other Directorship 8_x,Other Directorship 9_x,Other Directorship 10_x,Other Directorship 11_x,Other Directorship 12_x,Other Directorship 13_x,Other Directorship 14_x,Other Directorship 15_x,Brief Profile_x,Tenure Valid till_x,Indep_x,Appointment Date_x,Cessation Date_x,PrevLastServed_x,NextServed_x,CessationDummy_x,ReappointDummy_x,TermStartDummy_x,TermNumber_x,CumOpBalUnc_x,CumOpBalIndep_x,CumOpBalNonIndep_x,CumCloBalUnc_x,CumCloBalIndep_x,CumCloBalNonIndep_x,TermOpBalUnc_x,TermOpBalIndep_x,TermOpBalNonIndep_x,TermOpBalTotal_x,TermCloBalUnc_x,TermCloBalIndep_x,TermCloBalNonIndep_x,TermCloBalTotal_x,CompOpBalUnc_x,CompOpBalIndep_x,CompOpBalNonIndep_x,CompOpBalTotal_x,CompCloBalUnc_x,CompCloBalIndep_x,CompCloBalNonIndep_x,CompCloBalTotal_x,CloBalTotalXP_x,AllPastDirect_x,CurrDirectA_x,CurrDirectAB_x,PastSiezedDirectA_x,PastSiezedDirectAB_x,CountOtherPastUnclearA_x,CountOtherPastIndepA_x,CountOtherPastNonIndepA_x,CompCountOtherPastTotalA_x,CountOtherPastUnclearAB_x,CountOtherPastIndepAB_x,CountOtherPastNonIndepAB_x,CompCountOtherPastTotalAB_x,CountCurrUnclearA_x,CountCurrIndepA_x,CountCurrNonIndepA_x,CompCountCurrTotalA_x,CountCurrUnclearAB_x,CountCurrIndepAB_x,CountCurrNonIndepAB_x,CompCountCurrTotalAB_x,Rookie_x,IsIndep_x,IsNonIndep_x,IsRookie_x,IsNonRookie_x,IsCeoMDPosition_x,IsChairmanPosition_x,IsCeoMDOccupation_x,IsChairmanOccupation_x,IsCeoMD_x,IsChairman_x,IsPromoterClassification_x,IsPromoterBoard_x,IsPromoter_x,IsDualityChairmanMD_x,IsFamilyManager_x,IsFamilyChairman_x,IsFamilyChairmanAndCEO_x,IsRookieIndep_x,IsRookieNonIndep_x,IsNonRookieIndep_x,IsNonRookieNonIndep_x,IsFemale_x,Age_x,TenureInYearsinCompIndep_x,TenureInYearsinCompTotal_x,IsFirstTerm_x,IsFirstTermIndep_x,IsOneYear_x,IsOneYearIndep_x,IsTwoYear_x,IsTwoYearIndep_x,IsThreeYear_x,IsThreeYearIndep_x,IsRetires5y_x,IsTermLimitRetirement_x,IsDefaultTerm_x,IsBusy_x,IsTurnOver_x,HasRetires5y_x,HasTermLimitRetirement_x,IsTurnOverIndep_x,CompanyName_x,Prowess company code_x,Entity type_x,Entity type code_x,Incorporation year_x,Industry group_x,Industry group code_x,NIC name_x,NIC code_x,Ownership group code_x,Ownership group_x,Age group_x,NSE symbol_x,Head office address_x,Skills_x,EducationAll_x,IsMBA_x,IsPhD_x,AllPastDirectNIC_x,CurrDirectANIC_x,CurrDirectABNIC_x,AllNIC_x,AllNIC_Industry_x,HasFinanceXP_x,HasTechXP_x,HasRelatedIndustryXP_x,HasExecXP_x,PublicExecXPDummy_x,PrivateExecXPDummy_x,HasPublicExecXP_x,HasPrivateExecXP_x,SkillsInPositiononBoard_x,SkillsInOccupation_x,AllSkills_x,HasTechSkill_x,HasFinanceSkill_x,FirmsPast_x,NumSkills_x,NumFirmsPast_x,NumIndustryPast_x,HasCeoMDChairXP_x,HasConglomerateXP_x,General Ability Index_x,ProwessCode_x,ACP_x,pct_x,RF_x,RMRF_x,MF_x,SMB_x,HML_x,OLS120_intercept_x,OLS120_RMRF_x,OLS120_SMB_x,OLS120_HML_x,OLS120_r_squared_x,OLS120_adjusted_r_squared_x,OLS120_f_p_value_x,120CAR3_x,120CAR5_x,120CAR7_x,120CAR11_x,OLS150_intercept_x,OLS150_RMRF_x,OLS150_SMB_x,OLS150_HML_x,OLS150_r_squared_x,OLS150_adjusted_r_squared_x,OLS150_f_p_value_x,150CAR3_x,150CAR5_x,150CAR7_x,150CAR11_x,OLS180_intercept_x,OLS180_RMRF_x,OLS180_SMB_x,OLS180_HML_x,OLS180_r_squared_x,OLS180_adjusted_r_squared_x,OLS180_f_p_value_x,180CAR3_x,180CAR5_x,180CAR7_x,180CAR11_x,FirmID,AsOnYear_y,Company Name,Sales,Profit after tax,PBDITA,Return on total assets,Total assets,Operating profit of non-financial companies,Operating profit of financial

# --------------------------------------------------------------------------------------------

# PSM functions

In [12]:
def MeanDiffTtest(sample, subset):

    group1 = sample.loc[ sample["Logit_Z"] == 1, subset].copy()
    group2 = sample.loc[ sample["Logit_Z"] == 0, subset].copy()
    
    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)  # Welch’s t-test (default)

    print(subset, ":")
    print("T Statistic:", t_stat, " P Value:",p_value)
    print("Mean1:", group1.mean(), " Mean2:", group2.mean(), " Diff:", group1.mean() - group2.mean())
    print("N_1:", len(group1), "; N_2:", len(group2), "[unique = ", len(sample.loc[ sample["Logit_Z"] == 0, ["Symbol", "AsOnDate", "Person Code"]].drop_duplicates()), "]")

    return 0

In [13]:
def LogitReg(sample, exog_var):
    
    # Logit Regression
    endog = sample[["Logit_Z"]]
    exog = sample[exog_var]
    exog = sm.add_constant(exog)
    
    log_reg = sm.Logit(endog, exog).fit()

    propensityScores = log_reg.predict(exog)
    
    return propensityScores
    

In [14]:
# Func PSM replacement

def PsmReplac(sample, exog_var, subset):

    # Logit Regression
    sample["propensity_score"] = LogitReg(sample, exog_var)
    
    treated = sample.loc[ sample["Logit_Z"] == 1].copy()
    control = sample.loc[ sample["Logit_Z"] == 0].copy()
    
    # Fit Nearest Neighbors model on control group propensity scores
    nn = NearestNeighbors(n_neighbors=1, metric="euclidean")
    nn.fit(control[["propensity_score"]])
    
    # Find the nearest neighbor for each treated unit
    distances, indices = nn.kneighbors(treated[["propensity_score"]])
    
    # Retrieve matched control units
    matchedControl = control.iloc[indices.flatten()].copy()
    
    # Add back the treated observations
    matched = pd.concat([treated, matchedControl])
    
    # Reset index
    matched.reset_index(drop=True, inplace=True)


    # Mean difference and T Test
    MeanDiffTtest(matched, f"{subset}CAR3")
    print("\n")
    MeanDiffTtest(matched, f"{subset}CAR5")
    print("\n")
    MeanDiffTtest(matched, f"{subset}CAR7")
    print("\n")
    MeanDiffTtest(matched, f"{subset}CAR11")
    print("\n")

    return


In [15]:
# Func PSM non replacement
def PsmNonReplac(sample, exog_var, subset):

    # Logit Regression
    sample["propensity_score"] = LogitReg(sample, exog_var)

    # Separate treated and control groups
    treated = sample[sample['Logit_Z'] == 1].copy()
    control = sample[sample['Logit_Z'] == 0].copy()
    
    # Compute pairwise distances (absolute difference in propensity scores)
    dist_matrix = cdist(treated[['propensity_score']], control[['propensity_score']], metric='euclidean')
    
    # Match without replacement
    matched_indices = []
    used_control_indices = set()
    
    for i in range(len(treated)):
        # Get nearest control unit index that hasn't been used
        match_idx = np.argmin(dist_matrix[i])
        
        while match_idx in used_control_indices:  # Ensure it's not already matched
            dist_matrix[i, match_idx] = np.inf  # Temporarily set distance to infinity
            match_idx = np.argmin(dist_matrix[i])
        
        used_control_indices.add(match_idx)
        matched_indices.append(match_idx)
    
    # Retrieve matched control units
    matched_control = control.iloc[matched_indices].copy()
    
    # Combine matched treated and control units
    matched_data = pd.concat([treated.reset_index(drop=True), matched_control.reset_index(drop=True)])
    
    # Reset index
    matched_data.reset_index(drop=True, inplace=True)


    # Mean difference and T Test
    MeanDiffTtest(matched_data, f"{subset}CAR3")
    print("\n")
    MeanDiffTtest(matched_data, f"{subset}CAR5")
    print("\n")
    MeanDiffTtest(matched_data, f"{subset}CAR7")
    print("\n")
    MeanDiffTtest(matched_data, f"{subset}CAR11")
    print("\n")

    return
    


In [16]:
def MasterFunc(sample, exog_var):

    # With Replacement
    # 120 Day CAR
    print("WITH REPLACEMENT")
    print("120 Day CAR values")
    PsmReplac(sample.dropna(subset = ["120CAR3"]).reset_index(drop = True), exog_var, "120")
    # 150 Day CAR
    print("150 Day CAR values")
    PsmReplac(sample.dropna(subset = ["150CAR3"]).reset_index(drop = True), exog_var, "150")
    # 180 Day CAR
    print("180 Day CAR values")
    PsmReplac(sample.dropna(subset = ["180CAR3"]).reset_index(drop = True), exog_var, "180")

    # Without replacement 
    # 120 Day CAR
    print("WITHOUT REPLACEMENT")
    print("120 Day CAR values")
    PsmNonReplac(sample.dropna(subset = ["120CAR3"]).reset_index(drop = True), exog_var, "120")
    # 150 Day CAR
    print("150 Day CAR values")
    PsmNonReplac(sample.dropna(subset = ["150CAR3"]).reset_index(drop = True), exog_var, "150")
    # 180 Day CAR
    print("180 Day CAR values")
    PsmNonReplac(sample.dropna(subset = ["180CAR3"]).reset_index(drop = True), exog_var, "180")

    return

# --------------------------------------------------------------------------------------------

# ROOKIE VS NON ROOKIE -- ALL INDEP 

### Identifying Dep Variable [Rookie Indep Classification] for logit regression

In [17]:
carFinIndep = carFin.loc[ (carFin["IsRookieIndep"] == 1) | (carFin["IsNonRookieIndep"] == 1), :].reset_index(drop = True)
carFinIndep["Logit_Z"] = np.where( carFinIndep["IsRookieIndep"] == 1, 1, 0)
carFinIndep

,Symbol,Company,AsOnDate,AsOnYear_x,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,Prowess company code_x,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,NSE symbol_x,Head office address,Skills,EducationAll,IsMBA,IsPhD,AllPastDirectNIC,CurrDirectANIC,CurrDirectABNIC,AllNIC,AllNIC_Industry,HasFinanceXP,HasTechXP,HasRelatedIndustryXP,HasExecXP,PublicExecXPDummy,PrivateExecXPDummy,HasPublicExecXP,HasPrivateExecXP,SkillsInPositiononBoard,SkillsInOccupation,AllSkills,HasTechSkill,HasFinanceSkill,FirmsPast,NumSkills,NumFirmsPast,NumIndustryPast,HasCeoMDChairXP,HasConglomerateXP,General Ability Index,ProwessCode,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11,FirmID,AsOnYear_y,Company Name,Sales,Profit after tax,PBDITA,Return on total assets,Total assets,Operating profit of non-financial companies,Operating profit of financial companies,Debt to equity ratio (times),Long term borrowings incl current portion,Long term borrowings excl current portion,Debt,Trade payables (Old Sch. VI),Subscribed preference capital,Paid up preference capital (net of forfeited preference capital) (Old Sch. VI),Paid up preference capital (net of forfeited preference capital),Paid up preference shares,Total other receivables,Net cash flow from operating activities,Net cash inflow or (outf

# 1. Non govt firms 

In [18]:
# CUSTOM
sample = carFinIndep.loc[ carFinIndep["Ownership group code"] >= 20000000000, :].copy().reset_index(drop = True)
sample

,Symbol,Company,AsOnDate,AsOnYear_x,ISIN,Person Code,Director Salutation,Director First Name,Director Middle Name,Director Surname,Date of Birth,Gender,Nationality,Member of Civil Services,Promoter Director (Yes/No),Position on Board,Independent (Yes/No),Education1,Education2,Education3,Education4,Education5,Education6,Education7,Education8,Education9,Education10,Skills/Competencies,Occupation,Cessation Reason,Other Directorship 1,Other Directorship 2,Other Directorship 3,Other Directorship 4,Other Directorship 5,Other Directorship 6,Other Directorship 7,Other Directorship 8,Other Directorship 9,Other Directorship 10,Other Directorship 11,Other Directorship 12,Other Directorship 13,Other Directorship 14,Other Directorship 15,Brief Profile,Tenure Valid till,Indep,Appointment Date,Cessation Date,PrevLastServed,NextServed,CessationDummy,ReappointDummy,TermStartDummy,TermNumber,CumOpBalUnc,CumOpBalIndep,CumOpBalNonIndep,CumCloBalUnc,CumCloBalIndep,CumCloBalNonIndep,TermOpBalUnc,TermOpBalIndep,TermOpBalNonIndep,TermOpBalTotal,TermCloBalUnc,TermCloBalIndep,TermCloBalNonIndep,TermCloBalTotal,CompOpBalUnc,CompOpBalIndep,CompOpBalNonIndep,CompOpBalTotal,CompCloBalUnc,CompCloBalIndep,CompCloBalNonIndep,CompCloBalTotal,CloBalTotalXP,AllPastDirect,CurrDirectA,CurrDirectAB,PastSiezedDirectA,PastSiezedDirectAB,CountOtherPastUnclearA,CountOtherPastIndepA,CountOtherPastNonIndepA,CompCountOtherPastTotalA,CountOtherPastUnclearAB,CountOtherPastIndepAB,CountOtherPastNonIndepAB,CompCountOtherPastTotalAB,CountCurrUnclearA,CountCurrIndepA,CountCurrNonIndepA,CompCountCurrTotalA,CountCurrUnclearAB,CountCurrIndepAB,CountCurrNonIndepAB,CompCountCurrTotalAB,Rookie,IsIndep,IsNonIndep,IsRookie,IsNonRookie,IsCeoMDPosition,IsChairmanPosition,IsCeoMDOccupation,IsChairmanOccupation,IsCeoMD,IsChairman,IsPromoterClassification,IsPromoterBoard,IsPromoter,IsDualityChairmanMD,IsFamilyManager,IsFamilyChairman,IsFamilyChairmanAndCEO,IsRookieIndep,IsRookieNonIndep,IsNonRookieIndep,IsNonRookieNonIndep,IsFemale,Age,TenureInYearsinCompIndep,TenureInYearsinCompTotal,IsFirstTerm,IsFirstTermIndep,IsOneYear,IsOneYearIndep,IsTwoYear,IsTwoYearIndep,IsThreeYear,IsThreeYearIndep,IsRetires5y,IsTermLimitRetirement,IsDefaultTerm,IsBusy,IsTurnOver,HasRetires5y,HasTermLimitRetirement,IsTurnOverIndep,CompanyName,Prowess company code_x,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,NSE symbol_x,Head office address,Skills,EducationAll,IsMBA,IsPhD,AllPastDirectNIC,CurrDirectANIC,CurrDirectABNIC,AllNIC,AllNIC_Industry,HasFinanceXP,HasTechXP,HasRelatedIndustryXP,HasExecXP,PublicExecXPDummy,PrivateExecXPDummy,HasPublicExecXP,HasPrivateExecXP,SkillsInPositiononBoard,SkillsInOccupation,AllSkills,HasTechSkill,HasFinanceSkill,FirmsPast,NumSkills,NumFirmsPast,NumIndustryPast,HasCeoMDChairXP,HasConglomerateXP,General Ability Index,ProwessCode,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,120CAR3,120CAR5,120CAR7,120CAR11,OLS150_intercept,OLS150_RMRF,OLS150_SMB,OLS150_HML,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,150CAR3,150CAR5,150CAR7,150CAR11,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,180CAR3,180CAR5,180CAR7,180CAR11,FirmID,AsOnYear_y,Company Name,Sales,Profit after tax,PBDITA,Return on total assets,Total assets,Operating profit of non-financial companies,Operating profit of financial companies,Debt to equity ratio (times),Long term borrowings incl current portion,Long term borrowings excl current portion,Debt,Trade payables (Old Sch. VI),Subscribed preference capital,Paid up preference capital (net of forfeited preference capital) (Old Sch. VI),Paid up preference capital (net of forfeited preference capital),Paid up preference shares,Total other receivables,Net cash flow from operating activities,Net cash inflow or (outf

## 1.1 2013 - 2020

In [19]:
# CUSTOM
sample1 = sample.loc[ (sample["AsOnDate"] <= "2020-03-31") & (sample["AsOnDate"] >= "2013-03-31"), : ].copy().reset_index(drop = True)

### 1.1.1 Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [20]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.575794
         Iterations 5
120CAR3 :
T Statistic: 0.1152134059905395  P Value: 0.9082778169002441
Mean1: -0.0037544933850766275  Mean2: -0.003992165151409684  Diff: 0.00023767176633305654
N_1: 6870 ; N_2: 6870 [unique =  3983 ]


120CAR5 :
T Statistic: -1.8386290798054072  P Value: 0.0660002859250013
Mean1: -0.06283026690652527  Mean2: -0.05850765756630149  Diff: -0.004322609340223778
N_1: 6870 ; N_2: 6870 [unique =  3983 ]


120CAR7 :
T Statistic: -3.0658745814232486  P Value: 0.0021752850865122556
Mean1: -0.12140889622333201  Mean2: -0.11402914603696628  Diff: -0.007379750186365733
N_1: 6870 ; N_2: 6870 [unique =  3983 ]


120CAR11 :
T Statistic: -7.738075196151661  P Value: 1.0811528610879223e-14
Mean1: -0.2366488781617234  Mean2: -0.21510817910350744  Diff: -0.021540699058215956
N_1: 6870 ; N_2: 6870 [unique =  3983 ]


150 Day CAR values
O

### 1.1.2 Log Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [21]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.570259
         Iterations 5
120CAR3 :
T Statistic: -0.3086994034716006  P Value: 0.7575552476319898
Mean1: -0.0037544933850766275  Mean2: -0.003110881589463226  Diff: -0.0006436117956134016
N_1: 6870 ; N_2: 6870 [unique =  3976 ]


120CAR5 :
T Statistic: -2.8799347561131916  P Value: 0.003986141399106664
Mean1: -0.06283026690652527  Mean2: -0.05602426830610033  Diff: -0.0068059986004249395
N_1: 6870 ; N_2: 6870 [unique =  3976 ]


120CAR7 :
T Statistic: -4.741610948599721  P Value: 2.1439873085479383e-06
Mean1: -0.12140889622333201  Mean2: -0.1096681306408931  Diff: -0.011740765582438903
N_1: 6870 ; N_2: 6870 [unique =  3976 ]


120CAR11 :
T Statistic: -8.199868830200955  P Value: 2.6209910416290584e-16
Mean1: -0.2366488781617234  Mean2: -0.21336904742463883  Diff: -0.02327983073708456
N_1: 6870 ; N_2: 6870 [unique =  3976 ]


150 Day CAR values

### 1.1.3 Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [22]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.575887
         Iterations 5
120CAR3 :
T Statistic: -0.39020926331029415  P Value: 0.6963884105090947
Mean1: -0.0037544933850766275  Mean2: -0.002947299105322967  Diff: -0.0008071942797536606
N_1: 6870 ; N_2: 6870 [unique =  3974 ]


120CAR5 :
T Statistic: -2.845871636823328  P Value: 0.004438436624329451
Mean1: -0.06283026690652527  Mean2: -0.056175905387103556  Diff: -0.0066543615194217115
N_1: 6870 ; N_2: 6870 [unique =  3974 ]


120CAR7 :
T Statistic: -3.7502791129711386  P Value: 0.00017749447164372808
Mean1: -0.12140889622333201  Mean2: -0.11241253168156519  Diff: -0.008996364541766819
N_1: 6870 ; N_2: 6870 [unique =  3974 ]


120CAR11 :
T Statistic: -7.8101711938406515  P Value: 6.1325113629672866e-15
Mean1: -0.2366488781617234  Mean2: -0.21486721109260046  Diff: -0.021781667069122934
N_1: 6870 ; N_2: 6870 [unique =  3974 ]


150 Day CAR v

### 1.1.4 Log Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [23]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.572449
         Iterations 5
120CAR3 :
T Statistic: 1.27318303864949  P Value: 0.2029755499988925
Mean1: -0.0037544933850766275  Mean2: -0.007124644220626769  Diff: 0.0033701508355501414
N_1: 6870 ; N_2: 6870 [unique =  3992 ]


120CAR5 :
T Statistic: -0.31875591699188954  P Value: 0.7499164832538745
Mean1: -0.06283026690652527  Mean2: -0.061830291681610444  Diff: -0.000999975224914823
N_1: 6870 ; N_2: 6870 [unique =  3992 ]


120CAR7 :
T Statistic: -1.814858212895869  P Value: 0.06956759125089657
Mean1: -0.12140889622333201  Mean2: -0.11603427106683643  Diff: -0.005374625156495574
N_1: 6870 ; N_2: 6870 [unique =  3992 ]


120CAR11 :
T Statistic: -5.8254901461478985  P Value: 5.824879393723489e-09
Mean1: -0.2366488781617234  Mean2: -0.2177895301483427  Diff: -0.018859348013380683
N_1: 6870 ; N_2: 6870 [unique =  3992 ]


150 Day CAR values
Optimi

## 1.2 2013 - 2024

In [24]:
# CUSTOM
sample1 = sample.loc[ (sample["AsOnDate"] <= "2024-03-31") & (sample["AsOnDate"] >= "2013-03-31"), : ].copy().reset_index(drop = True)

### 1.2.1 Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [25]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.564956
         Iterations 6
120CAR3 :
T Statistic: 0.9699762379274491  P Value: 0.3320697961106507
Mean1: -0.00185940592609211  Mean2: -0.0032298697337749638  Diff: 0.0013704638076828537
N_1: 11023 ; N_2: 11023 [unique =  6397 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.052032659923308106  Mean2: -0.05442603106409704  Diff: 0.0023933711407889344
N_1: 11023 ; N_2: 11023 [unique =  6397 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.10158370175836472  Mean2: -0.10795142691820166  Diff: 0.00636772515983694
N_1: 11023 ; N_2: 11023 [unique =  6397 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.1993500155261975  Mean2: -0.20785522924684025  Diff: 0.008505213720642751
N_1: 11023 ; N_2: 11023 [unique =  6397 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.566165
         Iterati

### 1.2.2 Log Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [26]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.557215
         Iterations 5
120CAR3 :
T Statistic: 0.9460845321135668  P Value: 0.3441160575631902
Mean1: -0.00185940592609211  Mean2: -0.0032717260729356865  Diff: 0.0014123201468435765
N_1: 11023 ; N_2: 11023 [unique =  6350 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.052032659923308106  Mean2: -0.05519541094536581  Diff: 0.0031627510220577013
N_1: 11023 ; N_2: 11023 [unique =  6350 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.10158370175836472  Mean2: -0.10799325617879327  Diff: 0.006409554420428551
N_1: 11023 ; N_2: 11023 [unique =  6350 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.1993500155261975  Mean2: -0.2068333342205702  Diff: 0.007483318694372715
N_1: 11023 ; N_2: 11023 [unique =  6350 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.558295
         Iterati

### 1.2.3 Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [27]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.565127
         Iterations 5
120CAR3 :
T Statistic: 0.7255001517780911  P Value: 0.4681530910493388
Mean1: -0.00185940592609211  Mean2: -0.0029473657898874584  Diff: 0.0010879598637953483
N_1: 11023 ; N_2: 11023 [unique =  6388 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.052032659923308106  Mean2: -0.054057667224984396  Diff: 0.0020250073016762904
N_1: 11023 ; N_2: 11023 [unique =  6388 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.10158370175836472  Mean2: -0.10646864054898995  Diff: 0.004884938790625237
N_1: 11023 ; N_2: 11023 [unique =  6388 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.1993500155261975  Mean2: -0.20672651760966018  Diff: 0.007376502083462688
N_1: 11023 ; N_2: 11023 [unique =  6388 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.566343
         Itera

### 1.2.4 Log Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [28]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.559806
         Iterations 5
120CAR3 :
T Statistic: 1.49804688127126  P Value: 0.1341354041793597
Mean1: -0.00185940592609211  Mean2: -0.00429103129513843  Diff: 0.00243162536904632
N_1: 11023 ; N_2: 11023 [unique =  6346 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.052032659923308106  Mean2: -0.057188507808286104  Diff: 0.005155847884977999
N_1: 11023 ; N_2: 11023 [unique =  6346 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.10158370175836472  Mean2: -0.10991331965897488  Diff: 0.008329617900610167
N_1: 11023 ; N_2: 11023 [unique =  6346 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.1993500155261975  Mean2: -0.2092292691597133  Diff: 0.0098792536335158
N_1: 11023 ; N_2: 11023 [unique =  6346 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.561028
         Iterations 5
15

# 2. Non govt firms & isFamilyChairmanAndCEO == 1

In [29]:
# CUSTOM
nonGov = carFinIndep.loc[ carFinIndep["Ownership group code"] >= 20000000000, :].copy().reset_index(drop = True)
familyChairCEOFirm = car.loc[ car["IsFamilyChairmanAndCEO"] == 1, ["Symbol", "Company", "AsOnDate"]].drop_duplicates()

sample = familyChairCEOFirm.merge(nonGov, on = ["Symbol", "Company", "AsOnDate"], how = "left").reset_index(drop = True)

## 2.1 2013 - 2020

In [30]:
# CUSTOM
sample1 = sample.loc[ (sample["AsOnDate"] <= "2020-03-31") & (sample["AsOnDate"] >= "2013-03-31"), : ].copy().reset_index(drop = True)

### 2.1.1 Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [31]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.583851
         Iterations 5
120CAR3 :
T Statistic: -0.3644389401676192  P Value: 0.715546016564473
Mean1: -0.0006002562521099553  Mean2: 0.00012559232221605397  Diff: -0.0007258485743260092
N_1: 2483 ; N_2: 2483 [unique =  1420 ]


120CAR5 :
T Statistic: -1.7236022109026703  P Value: 0.08484274822666776
Mean1: -0.05876831858297043  Mean2: -0.05467874727072857  Diff: -0.0040895713122418575
N_1: 2483 ; N_2: 2483 [unique =  1420 ]


120CAR7 :
T Statistic: -1.5617961374559237  P Value: 0.11839998365845471
Mean1: -0.11633181947463235  Mean2: -0.11193872743632556  Diff: -0.0043930920383067895
N_1: 2483 ; N_2: 2483 [unique =  1420 ]


120CAR11 :
T Statistic: -5.293897999648074  P Value: 1.2506203397287698e-07
Mean1: -0.23282759576029421  Mean2: -0.2141252242278481  Diff: -0.01870237153244611
N_1: 2483 ; N_2: 2483 [unique =  1420 ]


150 Day CAR values


### 2.1.2 Log Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [32]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.576384
         Iterations 5
120CAR3 :
T Statistic: -1.6746917977458493  P Value: 0.09405820366241939
Mean1: -0.0006002562521099553  Mean2: 0.0027861000928048325  Diff: -0.003386356344914788
N_1: 2483 ; N_2: 2483 [unique =  1409 ]


120CAR5 :
T Statistic: -3.428604008430997  P Value: 0.0006116820071394175
Mean1: -0.05876831858297043  Mean2: -0.050595304903331126  Diff: -0.008173013679639303
N_1: 2483 ; N_2: 2483 [unique =  1409 ]


120CAR7 :
T Statistic: -3.4624752106116787  P Value: 0.0005397919816329186
Mean1: -0.11633181947463235  Mean2: -0.10677273000372878  Diff: -0.009559089470903573
N_1: 2483 ; N_2: 2483 [unique =  1409 ]


120CAR11 :
T Statistic: -5.886630206050911  P Value: 4.20791422770305e-09
Mean1: -0.23282759576029421  Mean2: -0.21193501912970938  Diff: -0.02089257663058483
N_1: 2483 ; N_2: 2483 [unique =  1409 ]


150 Day CAR values

### 2.1.3 Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [33]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.583851
         Iterations 5
120CAR3 :
T Statistic: 1.2736750929024403  P Value: 0.20283823834820813
Mean1: -0.0006002562521099553  Mean2: -0.003310981652343799  Diff: 0.0027107254002338437
N_1: 2483 ; N_2: 2483 [unique =  1410 ]


120CAR5 :
T Statistic: -1.2746669162426465  P Value: 0.20248709786845945
Mean1: -0.05876831858297043  Mean2: -0.05567994154837805  Diff: -0.003088377034592381
N_1: 2483 ; N_2: 2483 [unique =  1410 ]


120CAR7 :
T Statistic: -1.9278294804571867  P Value: 0.05393352187862866
Mean1: -0.11633181947463235  Mean2: -0.11091208826395041  Diff: -0.005419731210681941
N_1: 2483 ; N_2: 2483 [unique =  1410 ]


120CAR11 :
T Statistic: -5.505123609281215  P Value: 3.880412692212123e-08
Mean1: -0.23282759576029421  Mean2: -0.2133477899325673  Diff: -0.019479805827726904
N_1: 2483 ; N_2: 2483 [unique =  1410 ]


150 Day CAR values
Opt

### 2.1.4 Log Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [34]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.576635
         Iterations 5
120CAR3 :
T Statistic: -0.46169156514422577  P Value: 0.6443230537914908
Mean1: -0.0006002562521099553  Mean2: 0.00031927923288595195  Diff: -0.0009195354849959072
N_1: 2483 ; N_2: 2483 [unique =  1405 ]


120CAR5 :
T Statistic: -1.8718010494838504  P Value: 0.06129404223190295
Mean1: -0.05876831858297043  Mean2: -0.054378770440555495  Diff: -0.004389548142414934
N_1: 2483 ; N_2: 2483 [unique =  1405 ]


120CAR7 :
T Statistic: -2.494604105646344  P Value: 0.012642197939601712
Mean1: -0.11633181947463235  Mean2: -0.1093584584663262  Diff: -0.006973361008306145
N_1: 2483 ; N_2: 2483 [unique =  1405 ]


120CAR11 :
T Statistic: -5.650458759764484  P Value: 1.6925307521270104e-08
Mean1: -0.23282759576029421  Mean2: -0.21290576314530169  Diff: -0.01992183261499253
N_1: 2483 ; N_2: 2483 [unique =  1405 ]


150 Day CAR values

## 2.2 2013 - 2024

In [35]:
# CUSTOM
sample1 = sample.loc[ (sample["AsOnDate"] <= "2024-03-31") & (sample["AsOnDate"] >= "2013-03-31"), : ].copy().reset_index(drop = True)

### 2.2.1 Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [36]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.574165
         Iterations 5
120CAR3 :
T Statistic: 1.3963222530209072  P Value: 0.16265561099022402
Mean1: 0.002234518798189699  Mean2: 5.674585669360586e-05  Diff: 0.002177772941496093
N_1: 4100 ; N_2: 4100 [unique =  2346 ]


120CAR5 :
T Statistic: 2.3499187886543917  P Value: 0.018801278727335644
Mean1: -0.046646932387168015  Mean2: -0.05117147323014547  Diff: 0.004524540842977452
N_1: 4100 ; N_2: 4100 [unique =  2346 ]


120CAR7 :
T Statistic: 2.7445052149021842  P Value: 0.006073459995184894
Mean1: -0.09553239874128057  Mean2: -0.10174853964897183  Diff: 0.006216140907691264
N_1: 4100 ; N_2: 4100 [unique =  2346 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.19214533664057143  Mean2: -0.20008797745155746  Diff: 0.00794264081098603
N_1: 4100 ; N_2: 4100 [unique =  2346 ]


150 Day CAR values
Optimization terminated successfully.
   

### 2.2.2 Log Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [37]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.564221
         Iterations 5
120CAR3 :
T Statistic: 0.16191896541081482  P Value: 0.8713736801243606
Mean1: 0.002234518798189699  Mean2: 0.001979757415208493  Diff: 0.0002547613829812058
N_1: 4100 ; N_2: 4100 [unique =  2349 ]


120CAR5 :
T Statistic: 1.4334042182891151  P Value: 0.15178090401532413
Mean1: -0.046646932387168015  Mean2: -0.049388191042024104  Diff: 0.0027412586548560894
N_1: 4100 ; N_2: 4100 [unique =  2349 ]


120CAR7 :
T Statistic: 2.7263824174326885  P Value: 0.006416954028176523
Mean1: -0.09553239874128057  Mean2: -0.10168045076306025  Diff: 0.006148052021779676
N_1: 4100 ; N_2: 4100 [unique =  2349 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.19214533664057143  Mean2: -0.2025502169422463  Diff: 0.010404880301674857
N_1: 4100 ; N_2: 4100 [unique =  2349 ]


150 Day CAR values
Optimization terminated successfully.
  

### 2.2.3 Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [38]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.574223
         Iterations 5
120CAR3 :
T Statistic: 1.0369889698567096  P Value: 0.29977237923704114
Mean1: 0.002234518798189699  Mean2: 0.0006651378979876434  Diff: 0.0015693809002020553
N_1: 4100 ; N_2: 4100 [unique =  2353 ]


120CAR5 :
T Statistic: 2.689204544014848  P Value: 0.007177195016426983
Mean1: -0.046646932387168015  Mean2: -0.05162956350537706  Diff: 0.004982631118209048
N_1: 4100 ; N_2: 4100 [unique =  2353 ]


120CAR7 :
T Statistic: 3.4224559602505407  P Value: 0.0006236626168704029
Mean1: -0.09553239874128057  Mean2: -0.10305943523676103  Diff: 0.007527036495480463
N_1: 4100 ; N_2: 4100 [unique =  2353 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.19214533664057143  Mean2: -0.20388327610294557  Diff: 0.011737939462374142
N_1: 4100 ; N_2: 4100 [unique =  2353 ]


150 Day CAR values
Optimization terminated successfully.
 

### 2.2.4 Log Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [39]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.565836
         Iterations 5
120CAR3 :
T Statistic: 0.5659623615462117  P Value: 0.5714350073538119
Mean1: 0.002234518798189699  Mean2: 0.0013537307506192372  Diff: 0.0008807880475704616
N_1: 4100 ; N_2: 4100 [unique =  2313 ]


120CAR5 :
T Statistic: 2.74478532905529  P Value: 0.006068415265180782
Mean1: -0.046646932387168015  Mean2: -0.05186605430536887  Diff: 0.005219121918200856
N_1: 4100 ; N_2: 4100 [unique =  2313 ]


120CAR7 :
T Statistic: 4.265695748809013  P Value: 2.0153127451575256e-05
Mean1: -0.09553239874128057  Mean2: -0.10512999890569566  Diff: 0.00959760016441509
N_1: 4100 ; N_2: 4100 [unique =  2313 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.19214533664057143  Mean2: -0.20295733090777882  Diff: 0.010811994267207387
N_1: 4100 ; N_2: 4100 [unique =  2313 ]


150 Day CAR values
Optimization terminated successfully.
    

# 3. Non govt firms & IsFamilyManager == 1

In [42]:
# CUSTOM
nonGov = carFinIndep.loc[ carFinIndep["Ownership group code"] >= 20000000000, :].copy().reset_index(drop = True)
familyManaged = car.loc[ car["IsFamilyManager"] == 1, ["Symbol", "Company", "AsOnDate"]].drop_duplicates()

sample = familyManaged.merge(nonGov, on = ["Symbol", "Company", "AsOnDate"], how = "left").reset_index(drop = True)

## 3.1 2013 - 2020

In [43]:
# CUSTOM
sample1 = sample.loc[ (sample["AsOnDate"] <= "2020-03-31") & (sample["AsOnDate"] >= "2013-03-31"), : ].copy().reset_index(drop = True)

### 3.1.1 Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [44]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.585669
         Iterations 5
120CAR3 :
T Statistic: -0.522656069572122  P Value: 0.6012271894420955
Mean1: -0.00024691814534637626  Mean2: 0.0006010547649761006  Diff: -0.0008479729103224769
N_1: 4431 ; N_2: 4431 [unique =  2547 ]


120CAR5 :
T Statistic: -3.4321631786803324  P Value: 0.0006015813240710543
Mean1: -0.05949095380672163  Mean2: -0.053029508369965526  Diff: -0.006461445436756107
N_1: 4431 ; N_2: 4431 [unique =  2547 ]


120CAR7 :
T Statistic: -4.407383857921419  P Value: 1.0585897281353682e-05
Mean1: -0.11776903676161558  Mean2: -0.10765712817990311  Diff: -0.010111908581712473
N_1: 4431 ; N_2: 4431 [unique =  2547 ]


120CAR11 :
T Statistic: -6.880421338792178  P Value: 6.372511873507618e-12
Mean1: -0.23423145299286807  Mean2: -0.21445726712562238  Diff: -0.01977418586724569
N_1: 4431 ; N_2: 4431 [unique =  2547 ]


150 Day CAR valu

### 3.1.2 Log Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [45]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.579826
         Iterations 5
120CAR3 :
T Statistic: -0.29703892021598066  P Value: 0.7664439724067988
Mean1: -0.00024691814534637626  Mean2: 0.00023943136080954155  Diff: -0.0004863495061559178
N_1: 4431 ; N_2: 4431 [unique =  2530 ]


120CAR5 :
T Statistic: -2.2635331490501778  P Value: 0.023627510213505565
Mean1: -0.05949095380672163  Mean2: -0.05524921210899414  Diff: -0.004241741697727494
N_1: 4431 ; N_2: 4431 [unique =  2530 ]


120CAR7 :
T Statistic: -2.8215310421484845  P Value: 0.00479024152910612
Mean1: -0.11776903676161558  Mean2: -0.11154765927046978  Diff: -0.006221377491145805
N_1: 4431 ; N_2: 4431 [unique =  2530 ]


120CAR11 :
T Statistic: -6.796416131458044  P Value: 1.1433274293825322e-11
Mean1: -0.23423145299286807  Mean2: -0.2159802202509564  Diff: -0.018251232741911677
N_1: 4431 ; N_2: 4431 [unique =  2530 ]


150 Day CAR valu

### 3.1.3 Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [46]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.585667
         Iterations 5
120CAR3 :
T Statistic: 1.4887897178280547  P Value: 0.13657954317964655
Mean1: -0.00024691814534637626  Mean2: -0.002675816604803991  Diff: 0.002428898459457615
N_1: 4431 ; N_2: 4431 [unique =  2524 ]


120CAR5 :
T Statistic: -1.5585970268540936  P Value: 0.11912872255372949
Mean1: -0.05949095380672163  Mean2: -0.056606471801683086  Diff: -0.0028844820050385467
N_1: 4431 ; N_2: 4431 [unique =  2524 ]


120CAR7 :
T Statistic: -2.742462173142764  P Value: 0.006110568885915587
Mean1: -0.11776903676161558  Mean2: -0.11185625153113447  Diff: -0.00591278523048111
N_1: 4431 ; N_2: 4431 [unique =  2524 ]


120CAR11 :
T Statistic: -5.92951847719683  P Value: 3.1546953167576155e-09
Mean1: -0.23423145299286807  Mean2: -0.21835613734455087  Diff: -0.015875315648317206
N_1: 4431 ; N_2: 4431 [unique =  2524 ]


150 Day CAR values
O

### 3.1.4 Log Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [47]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.580411
         Iterations 5
120CAR3 :
T Statistic: 1.343422704304402  P Value: 0.1791707952574096
Mean1: -0.00024691814534637626  Mean2: -0.002432318578438879  Diff: 0.0021854004330925025
N_1: 4431 ; N_2: 4431 [unique =  2520 ]


120CAR5 :
T Statistic: -1.3290907539024477  P Value: 0.1838528626383054
Mean1: -0.05949095380672163  Mean2: -0.056976051487142826  Diff: -0.0025149023195788073
N_1: 4431 ; N_2: 4431 [unique =  2520 ]


120CAR7 :
T Statistic: -1.8307292438943188  P Value: 0.06717470295815353
Mean1: -0.11776903676161558  Mean2: -0.11367997799931279  Diff: -0.004089058762302797
N_1: 4431 ; N_2: 4431 [unique =  2520 ]


120CAR11 :
T Statistic: -6.269266627672127  P Value: 3.8002046131073016e-10
Mean1: -0.23423145299286807  Mean2: -0.2173232419083806  Diff: -0.01690821108448748
N_1: 4431 ; N_2: 4431 [unique =  2520 ]


150 Day CAR values
Opt

## 3.2 2013 - 2024

In [48]:
# CUSTOM
sample1 = sample.loc[ (sample["AsOnDate"] <= "2024-03-31") & (sample["AsOnDate"] >= "2013-03-31"), : ].copy().reset_index(drop = True)

### 3.2.1 Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [49]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.576926
         Iterations 6
120CAR3 :
T Statistic: 1.4487957171164014  P Value: 0.14741732604603786
Mean1: 0.0008192875965423109  Mean2: -0.0010195888948538103  Diff: 0.0018388764913961212
N_1: 7101 ; N_2: 7101 [unique =  4070 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.049382365561022955  Mean2: -0.05406257521625895  Diff: 0.004680209655235995
N_1: 7101 ; N_2: 7101 [unique =  4070 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.09840656332457752  Mean2: -0.1068261434962224  Diff: 0.008419580171644875
N_1: 7101 ; N_2: 7101 [unique =  4070 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.19679325691709598  Mean2: -0.20976691648369994  Diff: 0.012973659566603957
N_1: 7101 ; N_2: 7101 [unique =  4070 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.578450
         Iterations 6


### 3.2.2 Log Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [50]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.569013
         Iterations 5
120CAR3 :
T Statistic: 0.42190924272580005  P Value: 0.6730978281053567
Mean1: 0.0008192875965423109  Mean2: 0.00028407637383752216  Diff: 0.0005352112227047887
N_1: 7101 ; N_2: 7101 [unique =  4039 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.049382365561022955  Mean2: -0.05268475662774167  Diff: 0.0033023910667187156
N_1: 7101 ; N_2: 7101 [unique =  4039 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.09840656332457752  Mean2: -0.10753097548683586  Diff: 0.009124412162258344
N_1: 7101 ; N_2: 7101 [unique =  4039 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.19679325691709598  Mean2: -0.21053707614578576  Diff: 0.01374381922868978
N_1: 7101 ; N_2: 7101 [unique =  4039 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.570365
         Iterations 5

### 3.2.3 Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [51]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.577103
         Iterations 5
120CAR3 :
T Statistic: 1.2264858948169366  P Value: 0.22003632082263416
Mean1: 0.0008192875965423109  Mean2: -0.000804233332816856  Diff: 0.001623520929359167
N_1: 7101 ; N_2: 7101 [unique =  4035 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.049382365561022955  Mean2: -0.05423769613391318  Diff: 0.004855330572890226
N_1: 7101 ; N_2: 7101 [unique =  4035 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.09840656332457752  Mean2: -0.10715096696192798  Diff: 0.008744403637350462
N_1: 7101 ; N_2: 7101 [unique =  4035 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.19679325691709598  Mean2: -0.20771478207878147  Diff: 0.010921525161685486
N_1: 7101 ; N_2: 7101 [unique =  4035 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.578634
         Iterations 5
1

### 3.2.4 Log Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [52]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.569819
         Iterations 5
120CAR3 :
T Statistic: 1.6041856983121998  P Value: 0.1086961395200344
Mean1: 0.0008192875965423109  Mean2: -0.0011986332280979391  Diff: 0.00201792082464025
N_1: 7101 ; N_2: 7101 [unique =  4028 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.049382365561022955  Mean2: -0.054374921510094536  Diff: 0.004992555949071581
N_1: 7101 ; N_2: 7101 [unique =  4028 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.09840656332457752  Mean2: -0.10776701935881611  Diff: 0.009360456034238587
N_1: 7101 ; N_2: 7101 [unique =  4028 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.19679325691709598  Mean2: -0.2090005904771968  Diff: 0.012207333560100814
N_1: 7101 ; N_2: 7101 [unique =  4028 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.571336
         Iterations 5
15

# 4. Non govt firms & Non Family

In [54]:
# CUSTOM
nonGov = carFinIndep.loc[ carFinIndep["Ownership group code"] >= 20000000000, :].copy().reset_index(drop = True)
nonFamilyManaged = car.loc[ (car["IsFamilyManager"] == 0) & (car["IsFamilyChairmanAndCEO"] == 0), ["Symbol", "Company", "AsOnDate"]].drop_duplicates()

sample = nonFamilyManaged.merge(nonGov, on = ["Symbol", "Company", "AsOnDate"], how = "left").reset_index(drop = True)

## 4.1 2013 - 2020

In [55]:
# CUSTOM
sample1 = sample.loc[ (sample["AsOnDate"] <= "2020-03-31") & (sample["AsOnDate"] >= "2013-03-31"), : ].copy().reset_index(drop = True)

### 4.1.1 Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [56]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.575794
         Iterations 5
120CAR3 :
T Statistic: 0.13051829307229648  P Value: 0.896158475779471
Mean1: -0.003754493385076628  Mean2: -0.004023653770951611  Diff: 0.00026916038587498307
N_1: 6870 ; N_2: 6870 [unique =  3983 ]


120CAR5 :
T Statistic: -2.167192863349186  P Value: 0.030244578683784898
Mean1: -0.06283026690652527  Mean2: -0.05774878693821775  Diff: -0.005081479968307517
N_1: 6870 ; N_2: 6870 [unique =  3983 ]


120CAR7 :
T Statistic: -3.7465039604123214  P Value: 0.0001801839538039846
Mean1: -0.12140889622333201  Mean2: -0.11242891034119694  Diff: -0.00897998588213507
N_1: 6870 ; N_2: 6870 [unique =  3983 ]


120CAR11 :
T Statistic: -7.677278489170792  P Value: 1.737280743947363e-14
Mean1: -0.2366488781617234  Mean2: -0.21534538550709223  Diff: -0.021303492654631162
N_1: 6870 ; N_2: 6870 [unique =  3983 ]


150 Day CAR values
Opt

### 4.1.2 Log Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [57]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.570259
         Iterations 5
120CAR3 :
T Statistic: -0.6854414358548838  P Value: 0.49307791662003475
Mean1: -0.003754493385076628  Mean2: -0.0023283476941550092  Diff: -0.0014261456909216187
N_1: 6870 ; N_2: 6870 [unique =  3976 ]


120CAR5 :
T Statistic: -3.0042103300993355  P Value: 0.0026694062707072687
Mean1: -0.06283026690652527  Mean2: -0.05573279816109315  Diff: -0.0070974687454321145
N_1: 6870 ; N_2: 6870 [unique =  3976 ]


120CAR7 :
T Statistic: -4.328977566728556  P Value: 1.5109973963881778e-05
Mean1: -0.12140889622333201  Mean2: -0.11118741552175655  Diff: -0.010221480701575456
N_1: 6870 ; N_2: 6870 [unique =  3976 ]


120CAR11 :
T Statistic: -8.43767552020089  P Value: 3.568888448187608e-17
Mean1: -0.2366488781617234  Mean2: -0.21346273908479504  Diff: -0.023186139076928347
N_1: 6870 ; N_2: 6870 [unique =  3976 ]


150 Day CAR valu

### 4.1.3 Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [58]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.575887
         Iterations 5
120CAR3 :
T Statistic: -0.17038618358652896  P Value: 0.8647091729337077
Mean1: -0.003754493385076628  Mean2: -0.0034019634007534504  Diff: -0.0003525299843231775
N_1: 6870 ; N_2: 6870 [unique =  3974 ]


120CAR5 :
T Statistic: -3.0383921260349678  P Value: 0.0023847460786041003
Mean1: -0.06283026690652527  Mean2: -0.05567957436651147  Diff: -0.007150692540013798
N_1: 6870 ; N_2: 6870 [unique =  3974 ]


120CAR7 :
T Statistic: -3.8907708908132084  P Value: 0.00010049195974481063
Mean1: -0.12140889622333201  Mean2: -0.11214570974921675  Diff: -0.009263186474115254
N_1: 6870 ; N_2: 6870 [unique =  3974 ]


120CAR11 :
T Statistic: -7.904155845012046  P Value: 2.9075415195230244e-15
Mean1: -0.2366488781617234  Mean2: -0.2147450745703353  Diff: -0.021903803591388105
N_1: 6870 ; N_2: 6870 [unique =  3974 ]


150 Day CAR val

### 4.1.4 Log Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [59]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.572449
         Iterations 5
120CAR3 :
T Statistic: 1.2430354134434878  P Value: 0.21387716913162752
Mean1: -0.003754493385076628  Mean2: -0.007107845408734482  Diff: 0.003353352023657854
N_1: 6870 ; N_2: 6870 [unique =  3992 ]


120CAR5 :
T Statistic: -0.4941762471845622  P Value: 0.6211896650556921
Mean1: -0.06283026690652527  Mean2: -0.06127713782563779  Diff: -0.0015531290808874776
N_1: 6870 ; N_2: 6870 [unique =  3992 ]


120CAR7 :
T Statistic: -1.5062859072438486  P Value: 0.13201689657965562
Mean1: -0.12140889622333201  Mean2: -0.1169374173895261  Diff: -0.004471478833805914
N_1: 6870 ; N_2: 6870 [unique =  3992 ]


120CAR11 :
T Statistic: -5.531212457194683  P Value: 3.240151267364606e-08
Mean1: -0.2366488781617234  Mean2: -0.21846121676942065  Diff: -0.01818766139230274
N_1: 6870 ; N_2: 6870 [unique =  3992 ]


150 Day CAR values
Optimiz

## 4.2 2013 - 2024

In [60]:
# CUSTOM
sample1 = sample.loc[ (sample["AsOnDate"] <= "2024-03-31") & (sample["AsOnDate"] >= "2013-03-31"), : ].copy().reset_index(drop = True)

### 4.2.1 Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [61]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.564956
         Iterations 6
120CAR3 :
T Statistic: 0.7895777583054403  P Value: 0.4297832840611706
Mean1: -0.0018594059260921094  Mean2: -0.0030035152703723257  Diff: 0.0011441093442802162
N_1: 11023 ; N_2: 11023 [unique =  6397 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.052032659923308106  Mean2: -0.05415466600044975  Diff: 0.002122006077141643
N_1: 11023 ; N_2: 11023 [unique =  6397 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.10158370175836473  Mean2: -0.10613788373244304  Diff: 0.0045541819740783085
N_1: 11023 ; N_2: 11023 [unique =  6397 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.1993500155261975  Mean2: -0.20630250495790664  Diff: 0.006952489431709147
N_1: 11023 ; N_2: 11023 [unique =  6397 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.566165
         Iter

### 4.2.2 Log Market Cap, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [62]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Market Capitalisation", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.557215
         Iterations 5
120CAR3 :
T Statistic: 1.935425754521607  P Value: 0.05295084609585358
Mean1: -0.0018594059260921094  Mean2: -0.004969056261059311  Diff: 0.0031096503349672014
N_1: 11023 ; N_2: 11023 [unique =  6350 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.052032659923308106  Mean2: -0.05561615095839862  Diff: 0.003583491035090515
N_1: 11023 ; N_2: 11023 [unique =  6350 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.10158370175836473  Mean2: -0.10785656037202873  Diff: 0.006272858613664001
N_1: 11023 ; N_2: 11023 [unique =  6350 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.1993500155261975  Mean2: -0.20536315814744519  Diff: 0.006013142621247691
N_1: 11023 ; N_2: 11023 [unique =  6350 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.558295
         Iterat

### 4.2.3 Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [63]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.565127
         Iterations 5
120CAR3 :
T Statistic: 0.3725612389514155  P Value: 0.709478660179379
Mean1: -0.0018594059260921094  Mean2: -0.0024212191742318807  Diff: 0.0005618132481397713
N_1: 11023 ; N_2: 11023 [unique =  6388 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.052032659923308106  Mean2: -0.05346202139879284  Diff: 0.001429361475484732
N_1: 11023 ; N_2: 11023 [unique =  6388 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.10158370175836473  Mean2: -0.10563619918237827  Diff: 0.004052497424013535
N_1: 11023 ; N_2: 11023 [unique =  6388 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.1993500155261975  Mean2: -0.20606568151386176  Diff: 0.0067156659876642655
N_1: 11023 ; N_2: 11023 [unique =  6388 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.566343
         Itera

### 4.2.4 Log Total assets, ROtotalA, NonPromoHoldings-Institutions, DE ratio

In [64]:
# CUSTOM
logitCol = ["Symbol", "AsOnDate", "Person Code", "log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)", "Logit_Z"]

sampleReady = sample1.dropna( subset = logitCol ).reset_index(drop = True).copy()
exog_var = ["log_Total assets", "Return on total assets",
            "Non-promoter Institutions (In %) - Shares held", "Debt to equity ratio (times)"]

print("ROOKIE INDEP VS NON-ROOKIE INDEP")
MasterFunc(sampleReady, exog_var)

ROOKIE INDEP VS NON-ROOKIE INDEP
WITH REPLACEMENT
120 Day CAR values
Optimization terminated successfully.
         Current function value: 0.559806
         Iterations 5
120CAR3 :
T Statistic: 1.5425953542011286  P Value: 0.1229433798120611
Mean1: -0.0018594059260921094  Mean2: -0.004389826143408373  Diff: 0.0025304202173162637
N_1: 11023 ; N_2: 11023 [unique =  6346 ]


120CAR5 :
T Statistic: nan  P Value: nan
Mean1: -0.052032659923308106  Mean2: -0.05724160122789473  Diff: 0.0052089413045866215
N_1: 11023 ; N_2: 11023 [unique =  6346 ]


120CAR7 :
T Statistic: nan  P Value: nan
Mean1: -0.10158370175836473  Mean2: -0.10994506388781189  Diff: 0.008361362129447159
N_1: 11023 ; N_2: 11023 [unique =  6346 ]


120CAR11 :
T Statistic: nan  P Value: nan
Mean1: -0.1993500155261975  Mean2: -0.20992225372915319  Diff: 0.010572238202955692
N_1: 11023 ; N_2: 11023 [unique =  6346 ]


150 Day CAR values
Optimization terminated successfully.
         Current function value: 0.561028
         Itera